# Terminoloji Dosyası hazırlama

In [ ]:
!pip install requests PyMuPDF pandas torch transformers

In [ ]:
import requests
import fitz
import pandas as pd
import sqlite3
import re
from langdetect import detect  # Daha hızlı dil tespiti için langdetect kullanıyoruz

In [ ]:
import fitz
import pandas as pd
import re

class PDFPipeline:
    def __init__(self, pdf_path: str, csv_path: str):
        self.pdf_path = pdf_path
        self.csv_path = csv_path

    def extract_text_from_pdf(self) -> str:
        pdf_document = fitz.open(self.pdf_path)
        text = ""
        for page_num in range(9, 183):  # Belirli sayfa aralığını kontrol edin
            page = pdf_document.load_page(page_num)
            text += page.get_text("text")
        return text

    def process_text(self, text: str) -> pd.DataFrame:
        lines = text.split('\n')
        data = []
        current_english_term = []
        current_turkish_terms = []

        for line in lines:
            line = line.strip()
            if not line or re.match(r'Bilgisayar Mühendisleri Odası|Siber Güvenlik Terim Karşılıkları|^\d+$', line):
                continue

            # Yeni İngilizce terim olup olmadığını kontrol et
            if re.match(r'^[A-Z][A-Za-z ]+$', line):
                # Önceki İngilizce terimi ve Türkçe terimleri ekle
                if current_english_term:
                    if current_turkish_terms:
                        data.append([" ".join(current_english_term).strip(), " ".join(current_turkish_terms).strip()])
                    else:
                        # Eğer Türkçe terim yoksa İngilizce terim ile boş Türkçe terim ekle
                        data.append([" ".join(current_english_term).strip(), ""])
                # Yeni İngilizce terimi ayarla
                current_english_term = [line]
                current_turkish_terms = []
            else:
                # Türkçe terimleri biriktir
                current_turkish_terms.append(line)
                # Eğer İngilizce terim birden fazla kelimeden oluşuyorsa, onu güncelleştir
                if re.match(r'^[A-Z][A-Za-z]+$', line):
                    current_english_term.append(line)

        # Son terim çiftini ekle
        if current_english_term:
            if current_turkish_terms:
                data.append([" ".join(current_english_term).strip(), " ".join(current_turkish_terms).strip()])
            else:
                data.append([" ".join(current_english_term).strip(), ""])

        df = pd.DataFrame(data, columns=["English Term", "Turkish Term"])

        # Boş terimleri kaldır
        df = df.dropna()

        return df

    def save_to_csv(self, df: pd.DataFrame):
        df.to_csv(self.csv_path, index=False, encoding='utf-8')

    def run(self):
        text = self.extract_text_from_pdf()
        df = self.process_text(text)
        if not df.empty:
            self.save_to_csv(df)
            print(f"Data has been successfully saved to the CSV file: {self.csv_path}")
        else:
            print("No data found to save.")
        print(df.head())

In [ ]:
# Pipeline'i çalıştırma
# Örnek dosya yolları
pdf_file_path = "/Users/seymakerkuklu/Desktop/Siber-Guvenlik-Terim-Karsiliklari.pdf"  # Yerel PDF dosyasının yolu
csv_file_path = "/Users/seymakerkuklu/Desktop/Siber-Guvenlik-Terim-Karsiliklari.csv"    # Kaydedilecek CSV dosyasının yolu

# PDFPipeline sınıfını başlatma
pipeline = PDFPipeline(pdf_path=pdf_file_path, csv_path=csv_file_path)

# PDF dosyasını işleme ve veriyi CSV'ye kaydetme
pipeline.run()
